In [1]:
### Exercise 2.3 ###

In [30]:
%reset -f
import itertools
import numpy as np
%matplotlib notebook
#%matplotlib inline
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB
import networkx as nx

class StopExecution(Exception):
    def _render_traceback_(self):
        pass

In [32]:
# parameters
solveLPOnly = True # set False to solve as an IP

In [34]:
# Supply for each blood type
Supplies = {
    # Blood type i: supply
    1: 20,  # O
    2: 40,  # A
    3: 30,  # B
    4: 10   # AB
}

# Demand for each blood type
Demands = {
    # Blood type i: demand
    1: 5,   # O
    2: 30,  # A
    3: 20,  # B
    4: 45   # AB
}

# Compatibility matrix (arcs) - who can donate to whom
Compatibility = {
    # (donor, recipient): 1 if compatible, 0 if not
    (1, 1): 1,  # O -> O
    (1, 2): 1,  # O -> A
    (1, 3): 1,  # O -> B
    (1, 4): 1,  # O -> AB
    (2, 2): 1,  # A -> A
    (2, 4): 1,  # A -> AB
    (3, 3): 1,  # B -> B
    (3, 4): 1,  # B -> AB
    (4, 4): 1   # AB -> AB
}

# Versatility weights (how valuable it is to leave blood in reserve)
Weights = {
    1: 4,  # O is most versatile
    2: 2,  # A is moderately versatile
    3: 2,  # B is moderately versatile
    4: 1   # AB is least versatile
}


In [36]:
# Set up the optimization model
model = gp.Model('BloodDistribution')

# Decision variables: how much blood to transfer from donor i to recipient j
x = model.addVars(Compatibility.keys(), name="flow", lb=0)

In [38]:
# Objective function: maximize leftover blood with weight preferences
model.setObjective(
    gp.quicksum(Weights[i] * (Supplies[i] - gp.quicksum(Compatibility[i, j] * x[i, j] for j in Demands.keys() if (i, j) in Compatibility)) for i in Supplies.keys()), 
    GRB.MAXIMIZE
)

# Constraints: blood distributed from a type can't exceed its supply
model.addConstrs(
    (gp.quicksum(Compatibility[i, j] * x[i, j] for j in Demands.keys() if (i, j) in Compatibility) <= Supplies[i] for i in Supplies.keys()), 
    name="supply_constraint"
)

# Constraints: each demand must be met exactly
model.addConstrs(
    (gp.quicksum(Compatibility[i, j] * x[i, j] for i in Supplies.keys() if (i, j) in Compatibility) == Demands[j] for j in Demands.keys()), 
    name="demand_constraint"
)


{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>}

In [40]:
# Optimize the model
model.optimize()

# Check for optimal solution
if model.status != GRB.Status.OPTIMAL:
    print("***** Gurobi solve status:", model.status)
    print("***** This is a problem. Model does not have an optimal solution.")
    raise StopExecution

# Printing results in the requested format
print(" ")
print("***** Flows: ")

for (i, j) in Compatibility.keys():
    if Compatibility[i, j] == 1:  # Only print flows on valid arcs
        arcflow = str(round(x[i, j].X, 4))
        print(f"x[({i}, {j}), *] = {arcflow} capacity: N/A")  # Assuming no explicit capacity constraint in this model

# Show the total leftover blood by type
print("\nRemaining blood by type:")
for i in Supplies.keys():
    used_blood = sum(Compatibility[i, j] * x[i, j].X for j in Demands.keys() if (i, j) in Compatibility)
    leftover_blood = Supplies[i] - used_blood
    print(f"Blood type {i}: {leftover_blood:.2f} units")

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[x86] - Darwin 23.5.0 23F79)

CPU model: Intel(R) Core(TM) i7-8850H CPU @ 2.60GHz
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 8 rows, 9 columns and 18 nonzeros
Model fingerprint: 0xba8c6df1
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 4e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 4e+01]
Presolve removed 4 rows and 3 columns
Presolve time: 0.01s
Presolved: 4 rows, 6 columns, 11 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.5000000e+01   1.750000e+01   0.000000e+00      0s
       3   -0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.01 seconds (0.00 work units)
Optimal objective -0.000000000e+00
 
***** Flows: 
x[(1, 1), *] = 5.0 capacity: N/A
x[(1, 2), *] = 15.0 capacity: N/A
x[(1, 3), *] = 0.0 capacity: N/A
x[(1, 4), *] = 0.0 capacity: N/A
x